In [ ]:
!pip install pandas faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

In [ ]:
agencies = pd.read_csv('/content/Agencies.csv')
astronauts = pd.read_csv('/content/Astronauts.csv')
missions = pd.read_csv('/content/Missions.csv')
mission_crew = pd.read_csv('/content/Mission_Crew.csv')
spacecraft = pd.read_csv('/content/Spacecraft.csv')

In [ ]:
agencies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Agency_id           84 non-null     int64  
 1   Agency_name         84 non-null     object 
 2   Country             84 non-null     object 
 3   Founded_year        84 non-null     int64  
 4   Budget_billion_usd  84 non-null     float64
dtypes: float64(1), int64(2), object(2)
memory usage: 3.4+ KB


In [ ]:
mission_crew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1215 entries, 0 to 1214
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Mission_id    1215 non-null   int64 
 1   Astronaut_id  1215 non-null   int64 
 2   Role          1215 non-null   object
dtypes: int64(2), object(1)
memory usage: 28.6+ KB


In [ ]:
spacecraft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Spacecraft_id      760 non-null    int64  
 1   Spacecraft_name    760 non-null    object 
 2   Manufacturer       760 non-null    object 
 3   Type               760 non-null    object 
 4   Cost_million_usd   760 non-null    float64
 5   first_launch_year  760 non-null    int64  
 6   Mission_id         760 non-null    int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 41.7+ KB


In [ ]:
astronauts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Astronaut_id    99 non-null     int64 
 1   Name            99 non-null     object
 2   Nationality     99 non-null     object
 3   Gender          99 non-null     object
 4   Birth_year      99 non-null     int64 
 5   Missions_count  99 non-null     int64 
dtypes: int64(3), object(3)
memory usage: 4.8+ KB


In [ ]:
# I want to increase the number of astronauts to make it compatible with the number of missions and spacecraft
from faker import Faker
import random

fake = Faker()
additional_count = 600 - len(astronauts)
def generate_astronaut(start_id):
    return {
        "Astronaut_id": start_id,
        "Name": fake.name(),
        "Nationality": fake.country(),
        "Gender": random.choice(["Male", "Female"]),
        "Birth_year": np.random.randint(1940, 2000),
        "Missions_count": np.random.randint(0, 6)
    }

# Generate new astronauts
new_astronauts = [generate_astronaut(i) for i in range(len(astronauts)+1, 601)]

# Convert to DataFrame
new_astronauts_df = pd.DataFrame(new_astronauts)

# Append to your original
astronauts = pd.concat([astronauts, new_astronauts_df], ignore_index=True)

In [ ]:
# since I have added all these astronauts a lot of them are without missions and since the Astronaut_id is a foreign key in another table I will try to assign missions to those newly added astronauts

# Get all mission IDs
mission_ids = missions['Mission_id'].tolist()

# Generate crew assignments for new astronauts only
new_astronaut_ids = astronauts.loc[astronauts['Astronaut_id'] > 99, 'Astronaut_id'].tolist()

crew_records = []
roles = ["Commander", "Pilot", "Engineer", "Scientist", "Specialist"]

for astro_id in new_astronaut_ids:
    # Each astronaut can be in 0–3 missions
    num_assignments = np.random.randint(0, 4)
    assigned_missions = np.random.choice(mission_ids, size=num_assignments, replace=False)

    for mission_id in assigned_missions:
        crew_records.append({
            "Mission_id": mission_id,
            "Astronaut_id": astro_id,
            "Role": np.random.choice(roles)  # assign random role
        })

# Convert to DataFrame
new_mission_crew = pd.DataFrame(crew_records)

# Append to the original mission_crew table
mission_crew = pd.concat([mission_crew, new_mission_crew], ignore_index=True)

In [ ]:
# I will try to make this table have realistic data so we don’t want unrealistic records like a 25-year-old astronaut having done 6 missions the number of missions should correlate with their age

current_year = datetime.now().year
astronauts['age'] = current_year - astronauts['Birth_year']

def assign_missions(age):
    if age < 30:
        return np.random.randint(0, 2)
    elif age < 40:
        return np.random.randint(1, 4)
    elif age < 50:
        return np.random.randint(2, 6)
    elif age < 60:
        return np.random.randint(3, 7)
    else:
        return np.random.randint(1, 5)

astronauts['Missions_count'] = astronauts['age'].apply(assign_missions)

In [ ]:
# then drop the the age column
astronauts.drop(columns=['age'], inplace=True)

In [ ]:
astronauts.head(15)

,Astronaut_id,Name,Nationality,Gender,Birth_year,Missions_count
0,1,Neil Armstrong,USA,Male,1930,3
1,2,Buzz Aldrin,USA,Male,1930,2
2,3,Michael Collins,USA,Male,1930,4
3,4,Alan Shepard,USA,Male,1923,1
4,5,John Glenn,USA,Male,1921,3
5,6,Jim Lovell,USA,Male,1928,1
6,7,Sally Ride,USA,Female,1951,2
7,8,Shannon Lucid,USA,Female,1943,1
8,9,Peggy Whitson,USA,Female,1960,1
9,10,Victor Glover,USA,Male,1976,2


In [ ]:
# let's try to assign missions to new added astronaut where each astronaut has one consistent role across all missions and each mission has 3 to 5 crew member

roles = ["Commander", "Pilot", "Engineer", "Scientist", "Specialist"]
# probability of each role
role_distribution = [0.1, 0.2, 0.25, 0.25, 0.2]

# assigns one of those roles to every astronaut once only
astronauts["Role"] = np.random.choice(
    roles,
    size=len(astronauts),
    p=role_distribution
)

crew_records = []

# loop over every astronaut and look at how many missions (Missions_count) they should be in
# The index of the row (_)
# astro representing the row’s data
for _, astro in astronauts.iterrows():
    astro_id = astro['Astronaut_id']
    mission_count = astro['Missions_count']
    role = astro['Role']
    # randomly pick that many missions from the missions table
    assigned_missions = np.random.choice(
        missions['Mission_id'],
        size=mission_count,
        replace=False
    )
    # add them to the mission_crew table with their fixed role
    for mission_id in assigned_missions:
        crew_records.append({
            "Mission_id": mission_id,
            "Astronaut_id": astro_id,
            "Role": role  # fixed role
        })

mission_crew = pd.DataFrame(crew_records)


In [ ]:
#  enforce 3–5 astronauts per mission
final_records = []
for mission_id, group in mission_crew.groupby("Mission_id"):
    # If a mission has less than 3 astronauts add some extra random astronauts until it has 3
    if len(group) < 3:
        # Add more astronauts
        needed = 3 - len(group)
        extra_astronauts = astronauts.sample(needed)
        for _, astro in extra_astronauts.iterrows():
            final_records.append({
                "Mission_id": mission_id,
                "Astronaut_id": astro['Astronaut_id'],
                "Role": astro['Role']
            })
    elif len(group) > 5:
        # If a mission has more than 5 astronauts randomly cut it down to 5
        group = group.sample(5)
    # each row will become a dictionary where the key=coulmn and the values = cell values and finaly put them all in final_records list
    final_records.extend(group.to_dict(orient="records"))

mission_crew = pd.DataFrame(final_records)

In [ ]:
# drop role I dont need it any more in this table
astronauts.drop(columns=['Role'], inplace=True)

In [ ]:
# let's check on the new shape of the tabels
print("Astronauts Table Shape:", astronauts.shape)
print("Mission_Crew Table Shape:", mission_crew.shape)

Astronauts Table Shape: (600, 6)
Mission_Crew Table Shape: (1806, 3)


In [ ]:
mission_crew.head(15)

,Mission_id,Astronaut_id,Role
0,1,122,Specialist
1,1,330,Pilot
2,1,352,Engineer
3,3,269,Commander
4,3,78,Engineer
5,3,385,Scientist
6,4,129,Pilot
7,4,179,Scientist
8,4,430,Pilot
9,5,157,Scientist


In [ ]:
mission_crew.tail(15)

,Mission_id,Astronaut_id,Role
1791,495,81,Engineer
1792,496,209,Pilot
1793,496,567,Specialist
1794,496,6,Pilot
1795,497,579,Scientist
1796,497,78,Engineer
1797,497,436,Specialist
1798,498,75,Specialist
1799,498,235,Scientist
1800,498,330,Pilot


In [ ]:
missions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Mission_id     499 non-null    int64  
 1   Mission_name   499 non-null    object 
 2   Destination    499 non-null    object 
 3   Mission_type   499 non-null    object 
 4   Status         499 non-null    object 
 5   launch_date    499 non-null    object 
 6   duration_days  494 non-null    float64
 7   Agency_id      499 non-null    int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 31.3+ KB


In [ ]:
# let's convert the launch_date into date type
missions['launch_date'] = pd.to_datetime(missions['launch_date'], format='%d/%m/%Y')

In [ ]:
# I will replace the Ongoing status with 'Completed' or 'Failed' randomly; since the Ongoing mission will give me a lot of unrealistic date data

# Function to randomly assign with 80% Completed, 20% Failed
def replace_status():
    return np.random.choice(['Completed', 'Failed'], p=[0.8, 0.2])

missions.loc[missions['Status'] == 'Ongoing', 'Status'] = missions.loc[missions['Status'] == 'Ongoing', 'Status'].apply(lambda x: replace_status())

In [ ]:
# for "Completed" and "Failed" missions the launch_date should be in the past not in the future so let's check on the data

# Filter missions with unrealistic future dates
today = pd.Timestamp.today()
bad_dates = missions[
    (missions['Status'].isin(['Completed', 'Failed'])) &
    (missions['launch_date'] > today)
]
print(f"Number of missions with unrealistic dates: {len(bad_dates)}")
display(bad_dates)

Number of missions with unrealistic dates: 0


,Mission_id,Mission_name,Destination,Mission_type,Status,launch_date,duration_days,Agency_id


In [ ]:
# Since there is no error data let's move on to the duration column I will try to put more realistic data since there is no way that the mission to the Moon has the same or greater time than the mission to Mars

duration_map = {
    'Low Earth Orbit': (1, 14), 'ISS': (1, 14), 'Tiangong Space Station': (1, 14), 'LEO': (1, 14), 'Earth Orbit': (1, 14), 'Suborbital': (1, 3),
    'Moon': (5, 30), 'Cislunar': (5, 30),
    'High Earth Orbit': (3, 20), 'Medium Earth Orbit': (3, 20), 'Sun-Synchronous Orbit': (3, 20), 'Geostationary Orbit': (3, 20),
    'Mars': (200, 700),
    'Jupiter': (400, 1000), 'Jupiter System': (400, 1000),
    'Saturn': (1000, 2000),
    'Pluto': (2000, 4000), 'Pluto and Kuiper Belt': (2000, 4000),
    'Deep Space': (3000, 5000), 'Interstellar Space': (3000, 5000),
    'Vesta and Ceres': (500, 1500), 'Asteroid Belt': (500, 1500),
    'Asteroid Itokawa': (500, 1500), 'Asteroid Ryugu': (500, 1500), 'Asteroid Bennu': (500, 1500),
    'Comet 67P': (500, 1500), 'Comet 67P/Churyumov–Gerasimenko': (500, 1500), 'Comet Halley': (500, 1500),
    'Venus': (100, 500), 'Mercury': (100, 500),
    'Sun': (50, 500), 'Sun-Earth L1': (50, 500), 'Sun-Earth L2': (50, 500), 'Heliocentric Orbit': (50, 500),
    'Inclined Geosynchronous Orbit': (3, 20)
}

# Assign durations
def assign_duration(row): # row series containing the data of one mission
    if row['Status'] == 'Planned':
        return 0   # always 0 for planned since the mission is in the future

    if row['Status'] == 'Completed':
        if row['Destination'] in duration_map:
            low, high = duration_map[row['Destination']]
            return np.random.randint(low, high+1)
        else:
            return np.random.randint(1, 365)  # default if unknown

    # If status == Failed → leave as it is
    return row['duration_days']

# Apply duration fix
missions['duration_days'] = missions.apply(assign_duration, axis=1)

In [ ]:
missions.sample(15)

,Mission_id,Mission_name,Destination,Mission_type,Status,launch_date,duration_days,Agency_id
365,366,Dream Chaser CRS-2 Mission 1,Low Earth Orbit,Space Station,Planned,2024-12-15,0.0,20
495,496,Lunar Lander Demo,Moon,Lunar Mission,Planned,2029-04-15,0.0,33
302,303,Artemis III,Moon,Manned,Planned,2026-09-20,0.0,1
444,445,Tianzhou-1,Low Earth Orbit,Space Station,Completed,2017-04-20,2.0,25
305,306,HTV-X1,ISS,Cargo Resupply,Failed,2024-01-10,60.0,6
338,339,VSS Enterprise Glide Flight 2,Suborbital,Test/Prototype,Completed,2010-11-04,3.0,18
59,60,XMM-Newton,High Earth Orbit,Space Telescope,Failed,1999-12-10,9000.0,2
208,209,AGILE,Low Earth Orbit,Space Telescope,Completed,2007-04-23,9.0,10
100,101,Mars 3,Mars,Planetary Exploration,Completed,1971-05-28,277.0,3
455,456,CBERS-1,Sun-Synchronous Orbit,Earth Observation,Completed,1999-10-14,11.0,27


In [ ]:
# let's check on null values
print(missions.isnull().sum())
print('-'*20)
print(spacecraft.isnull().sum())
print('-'*20)
print(astronauts.isnull().sum())
print('-'*20)
print(mission_crew.isnull().sum())
print('-'*20)
print(agencies.isnull().sum())

Mission_id       0
Mission_name     0
Destination      0
Mission_type     0
Status           0
launch_date      0
duration_days    0
Agency_id        0
dtype: int64
--------------------
Spacecraft_id        0
Spacecraft_name      0
Manufacturer         0
Type                 0
Cost_million_usd     0
first_launch_year    0
Mission_id           0
dtype: int64
--------------------
Astronaut_id      0
Name              0
Nationality       0
Gender            0
Birth_year        0
Missions_count    0
dtype: int64
--------------------
Mission_id      0
Astronaut_id    0
Role            0
dtype: int64
--------------------
Agency_id             0
Agency_name           0
Country               0
Founded_year          0
Budget_billion_usd    0
dtype: int64


In [ ]:
agencies.to_csv("agencies.csv", index=False)
missions.to_csv("missions.csv", index=False)
spacecraft.to_csv("spacecraft.csv", index=False)
astronauts.to_csv("astronauts.csv", index=False)
mission_crew.to_csv("mission_crew.csv", index=False)